In [1]:
import numpy as np
import keras
from keras.layers import Input,Add,Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model

Using TensorFlow backend.


In [2]:
from keras import backend as K

In [3]:
from keras.initializers import glorot_uniform

In [4]:
# Define a function for identity block. Here, Input activation and output activation should have same shape. (In our)
# example input(a[l]) and output a[l+3] has the same activations.

def identity_block(X,f,filters,stage,block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut=X
    
    X=Conv2D(F1, kernel_size=(1,1), strides=(1,1), padding='valid', name=conv_name_base+'2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
    X=Activation('relu')(X)
    
    X = Conv2D(filters=F2, kernel_size=(f,f),strides=(1,1),padding='same', name=conv_name_base + '2b',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F3, kernel_size=(1,1),strides=(1,1),padding='valid', name=conv_name_base + '2c',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    
    X=Add()([X,X_shortcut])
    X=Activation('relu')(X)
    
    return X
    
    

In [5]:
# Define Convolutional Block. It is same as identity block but there is Conv2D layer and BatchNorm Layer in the 
# shortcut path. Its used when input and output don't have same shape of activations. So, with conv layer, shape
# is made to that.

def convolutional_block(X,f,filters,stage,block,s=2):
    
    #def name base
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    #Getting filter size
    F1, F2, F3 = filters
    
    #for shortcut
    X_shortcut=X
    
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters=F2, kernel_size=(f,f),strides=(1,1),padding='same', name=conv_name_base + '2b',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    # Third component of main path 
    X = Conv2D(filters=F3, kernel_size=(1,1),strides=(1,1),padding='valid', name=conv_name_base + '2c',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    
    #shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1,1),strides=(s,s),padding='valid', name=conv_name_base + '1',kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)
    
    X=Add()([X,X_shortcut])
    X=Activation('relu')(X)
    
    return X

In [6]:
# Building resnet Model

def resnet(input_shape=(64,64,3), classes=6):
    
    # Defining input as a tensor
    X_input=Input(input_shape)
    
    X=ZeroPadding2D((3,3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    
    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128,128,512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128,128,512], stage=3, block='b')
    X = identity_block(X, 3, [128,128,512], stage=3, block='c')
    X = identity_block(X, 3, [128,128,512], stage=3, block='d')
    
    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    #stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    
    X = AveragePooling2D(pool_size=(2, 2),padding='same',name='avg_pool')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet')
    
    return model

In [7]:
model=resnet(input_shape=(64,64,3),classes=6)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Load dataset
import h5py
train_dataset = h5py.File('train_signs.h5', "r")
train_set_x_orig = np.array(train_dataset["train_set_x"][:]) 
train_set_y_orig = np.array(train_dataset["train_set_y"][:]) 

test_dataset = h5py.File('test_signs.h5', "r")
test_set_x_orig = np.array(test_dataset["test_set_x"][:]) 
test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels


In [11]:
# data preprocessing
X_train=train_set_x_orig/255
X_test=test_set_x_orig/255
y_train=np.eye(6)[train_set_y_orig.reshape(-1)]
y_test=np.eye(6)[test_set_y_orig.reshape(-1)]

In [12]:
X_train.shape, y_train.shape

((1080, 64, 64, 3), (1080, 6))

In [13]:
X_test.shape, y_test.shape

((120, 64, 64, 3), (120, 6))

In [14]:
#Visualizing data
import matplotlib.pyplot as plt
fig, subaxes=plt.subplots(3,3,figsize=(10,10))
index=np.array(np.random.permutation(range(1,1000))[1:10]).reshape([3,3])
for axesrow,indexrow in zip(subaxes,index):
    for axescolumn, indexcolumn in zip(axesrow,indexrow):
        axescolumn.imshow(X_train[indexcolumn])
        y="index: ", indexcolumn, " Label:", np.argmax(y_train[indexcolumn])
        axescolumn.set_title(y)
        axescolumn.get_xaxis().set_visible(False)
        axescolumn.get_xaxis().set_visible(False)

In [15]:
model.fit(X_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5
1080/1080 [==============================] - 148s 137ms/step - loss: 2.4677 - acc: 0.2917
Epoch 2/5
1080/1080 [==============================] - 141s 130ms/step - loss: 1.1399 - acc: 0.5370
Epoch 3/5
1080/1080 [==============================] - 145s 135ms/step - loss: 0.7593 - acc: 0.7407
Epoch 4/5
1080/1080 [==============================] - 149s 138ms/step - loss: 0.5005 - acc: 0.8315
Epoch 5/5
1080/1080 [==============================] - 141s 130ms/step - loss: 0.2560 - acc: 0.9231
